In [30]:
import pandas as pd
import numpy as np
from datetime import datetime

import fxcmpy
from apscheduler.schedulers.blocking import BlockingScheduler
import os

os_path = os.getcwd()

def z(df):
    return (df - df.mean()) / df.std()

sched = BlockingScheduler()
lot = 100
tf = 'm1'

@sched.scheduled_job('interval', seconds=60)
def timed_job():
    print('new time job')
    con = fxcmpy.fxcmpy(config_file = 'fxcm.cfg')
    if (len(con.get_open_positions()) > 0):
        check_position(con)
    trading(con)
    
def check_position(con):
    i = 0
    for i in range(len(con.open_pos)):
        print('i = %i' % i)
        trade_id = con.get_open_trade_ids()[i]
        pos = con.get_open_position(trade_id)
        amount = pos.get_amount()
        symbol = pos.get_currency()
        data = con.get_candles(symbol, period=tf, number=100)
        if (pos.get_isBuy()):
            data['Close_z'] = z(data['bidclose'])
            print(data.iloc[-1]['Close_z'])
            if data.iloc[-1]['Close_z'] > -1:
                con.close_trade(trade_id=trade_id, amount=amount)
                print('close %s' % symbol)
            else:
                print('no close')
        else:
            data['Close_z'] = z(data['askclose'])
            print(data.iloc[-1]['Close_z'])
            if data.iloc[-1]['Close_z'] < 1:
                con.close_trade(trade_id=trade_id, amount=amount)
                print('close %s id: %i amount: %i' % (symbol, trade_id, amount))
            else:
                print('no close')
        i += 1
            

    
def trading(con):
    
    symbols = ['EUR/USD', 'USD/JPY', 'GBP/USD', 'USD/CAD', 'USD/CHF', 'AUD/USD', 'NZD/USD', 'EUR/AUD', 'EUR/CAD', 'EUR/CHF', 'EUR/GBP', 'EUR/JPY', 'EUR/NZD', 'GBP/AUD', 'GBP/CAD', 'GBP/CHF', 'GBP/JPY', 'GBP/NZD', 'AUD/CAD', 'AUD/CHF', 'AUD/JPY', 'AUD/NZD', 'NZD/CAD', 'NZD/CHF', 'NZD/JPY', 'CAD/CHF', 'CAD/JPY', 'CHF/JPY']
    df = dict()
    for symbol in symbols:
        df[symbol] = con.get_candles(symbol, period=tf, number=100)
        df[symbol]['bid_Close_z'] = z(df[symbol]['bidclose'])
        df[symbol]['ask_Close_z'] = z(df[symbol]['askclose'])
        if df[symbol].iloc[-1]['ask_Close_z'] < -2:
            print('buy signal')
            if not hasCurrency(con, symbol):
                order = con.create_market_buy_order(symbol, lot)
                if order:
                    print('buy %s success' % symbol)
                else:
                    print('buy %s not success' % symbol)
        elif df[symbol].iloc[-1]['bid_Close_z'] > 2:
            print('sell signal')
            if not hasCurrency(con, symbol):
                order = con.create_market_sell_order(symbol, lot)
                if order:
                    print('sell %s success' % symbol)
                else:
                    print('sell %s not success' % symbol)
    print('---------')

def hasCurrency(con, symbol):
    i = 0
    for i in range(len(con.open_pos)):
        trade_id = con.get_open_trade_ids()[i]
        pos = con.get_open_position(trade_id)
        if symbol == pos.get_currency():
            print('have position')
            return True
    print('%i not have position %s' % (i, symbol))
    return False
    

#@sched.scheduled_job('cron', day_of_week='mon-fri', hour=10)
#def scheduled_job():
#    print('This job is run every weekday at 10am.')


In [31]:
if __name__ == "__main__":  
    sched.start()

new time job
i = 0
0.7978536928573374
close AUD/NZD
i = 1
1.5266302017791826
no close
i = 2
-1.2989536639930768
no close
i = 3
new time job
i = 0
1.4061735055537334
no close
i = 1
1.4492586139922297
no close
i = 2
-1.2831909916506514
no close
buy signal
2 not have position NZD/USD
buy NZD/USD success
sell signal
3 not have position EUR/NZD
sell EUR/NZD success
sell signal
4 not have position GBP/NZD
sell GBP/NZD success
buy signal
5 not have position AUD/CHF
buy AUD/CHF success
buy signal
6 not have position NZD/CHF
buy NZD/CHF success
buy signal
7 not have position NZD/JPY
buy NZD/JPY success
buy signal
8 not have position CAD/CHF
buy CAD/CHF success
---------
new time job
i = 0
-1.2973364780543466
no close
i = 1
-1.9090918066143945
no close
i = 2
-2.193316639780508
no close
i = 3
1.8567093824748286
no close
i = 4
1.6091676429586779
no close
i = 5
-2.2676829137599324
no close
i = 6
1.636805074708853
no close
i = 7
1.5416974786622015
no close
i = 8
-1.2549013394937127
no close
i = 9
-1

KeyboardInterrupt: 